# Classification and Regression Tree (CART)

### É proposto a solução de um problema de classificação por meio do algoritmo CART para indução de árvores de decisão. Será analisada uma base de dados contendo informações de fotos de cédulas de dinheiro em forma de 5 atributos que permitem dizer se uma cédula é genuína ou forjada. Essa base de dados pode ser obtida no repositório [UCI Machine Learning Repository](http://archive.ics.uci.edu/ml/datasets/banknote+authentication).

### Bibliotecas a serem utilizadas

In [1]:
from random import seed
from random import randrange
from csv import reader
import pandas as pd

### Base de dados

In [2]:
df = pd.read_csv('data_banknote_authentication_dataset.csv', names = ['variance', 'skewness', 'curtosis', 'entropy', 'class'])
df

,variance,skewness,curtosis,entropy,class
0,3.62160,8.66610,-2.8073,-0.44699,1
1,4.54590,8.16740,-2.4586,-1.46210,1
2,3.86600,-2.63830,1.9242,0.10645,1
3,3.45660,9.52280,-4.0112,-3.59440,1
4,0.32924,-4.45520,4.5718,-0.98880,1
...,...,...,...,...,...
1367,0.40614,1.34920,-1.4501,-0.55949,2
1368,-1.38870,-4.87730,6.4774,0.34179,2
1369,-3.75030,-13.45860,17.5932,-2.77710,2
1370,-3.56370,-8.38270,12.3930,-1.28230,2


### Função para importação dos dados

In [3]:
def load_csv(filename):
    file = open(filename)
    lines = reader(file)
    dataset = list(lines)
    return dataset

### Pré-processamento dos dados

Conversão de coluna string para float

In [4]:
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

Divisão dos dados em k partições para cross validation

In [5]:
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

### Divisão (particionamento) dos dados 
A divisão dos dados no algoritmo CART consiste na geração de duas listas de dados por meio de uma operação condicional com relação a um atributo dos dados

Função para cálculo de coeficiente Gini para um conjunto de sub-nós (grupos) e valores de classe conhecidos

In [6]:
def gini_index(groups, classes):
    # Contagem do total de amostras no ponto de divisão (split point)
    n_instances = float(sum([len(group) for group in groups]))
    # Soma dos coeficientes Gini ponderados para cada grupo
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # Condicional de segurança contra divisão por zero
        if size == 0:
            continue
        score = 0 
        # Cálculo da frequência de cada valor de classe (p) e do score p^2 
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size # .count() retorna o número de ocorrências de class_val 
            score += p*p # soma dos quadrados das frequências de cada classe no grupo
        # Ponderar o score do grupo pelo seu tamanho relativo
        gini += (1.0 - score)*(size / n_instances) # Quantifica o ganho gerado pela divisão
    return gini

Função para o split dos dados

In [7]:
def test_split(index, value, dataset):
    left, right = list(), list() # Lista para armazenamento dos dados dos sub-nós
    for row in dataset:
        if row[index] < value: # Critério de divisão
            left.append(row)
        else:
            right.append(row)
    return left, right # Retorna as listas de dados para cada sub-nó gerado

### Avaliação de todas as divisões possíveis dos dados
Com a função para o cálculo do coeficiente the Gini e a função test_split agora temos o necessário para avaliar as divisões. Dado a base de dados, devemos checar todo valor em cada atributo como um candidato de divisão, avaliar o custo da divisão e determinar a melhor divisão possível que se pode realizar. Uma vez que a melhor divisão é determinada, podemos utilizá-la na árvore de decisão.

Função para determinação da melhor divisão

In [8]:
def get_split(dataset):
    class_values = list(set(row[-1] for row in dataset)) # set() não permite valores repetidos
    b_index, b_value, b_score, b_groups = 9999, 9999, 9999, None
    for index in range(len(dataset[0])-1): # Não contar a coluna de classes
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            #print('X%d < %.3f Gini=%.3f' % ((index+1), row[index], gini))
            if gini < b_score: # Quanto menor o gini melhor
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index': b_index, 'value': b_value, 'groups': b_groups}

### Contrução da árvore de decisão
Para gerar o nó raiz da árvore basta aplicarmos get_split() para o dataset original. Já para gerar os sub-nós subsequentes é necessário aplicar get_split() de forma recursiva para cada sub-nó gerado. Desta forma, é necessário determinar o momento de interromper o crescimento da árvore.Para isso iremos utilizar o Maximum Tree Depth (máximo número de nós permitidos) na árvore e o Minimum Node Records (número mínimo de dados presentes em um nó). Uma outra condição que se deve levar em conta também é o caso em que todos os dados em um nó pertencem à mesma classe e nó não pode ser mais dividido. Quando interrompemos o crescimento da árvore em um dado nó, esse nó é chamado de nó terminal e ele é utilizado para realizar a predição final. Isso é feito de forma a escolher a classe mais frequente no grupo de dados no nó.

Função para seleção de uma classe em um nó terminal. Retorna a classe mais comum no nó

In [9]:
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count) # Retorna o item mais comum na lista

Função a realização do split recursivo dos nós. Cria sub-nós filhos de um nó ou declara o nó como terminal

In [10]:
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    # Checa se algum dos nós criados está vazio
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # Checa se max_depth foi atingida
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # Split do nó esquerdo (left child)
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    # Split do nó direito (right child)
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

Função para construção da árvore

In [11]:
def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

Função para print da árvore

In [12]:
def print_tree(node, depth=0):
    if isinstance(node, dict): # Checa se node é um dicionário
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

### Classificação de dados pela árvore de decisão

Função que faz predição de um dado com a árvore de decisão gerada

In [13]:
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

Algoritmo CART 

In [14]:
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return predictions

Função para cálculo da acurácia do modelo de classificação

In [15]:
def accuracy_metric(actual, predicted):
    correct = 0 
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100

Função para avaliar o modelo de classificação por meio de cross-validation

In [16]:
def evaluate_algorithm(dataset, algorithm, n_folds, *args): # *args é usado caso possa adicionar mais parâmetros na função
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

### Main()

In [17]:
# Teste do algoritmo CART para dados de autentiação de cédulas
seed(1)
# Importação e preparação dos dados
filename = 'data_banknote_authentication_dataset.csv'
dataset = load_csv(filename)
# Conversão de atributos string para float
for i in range(len(dataset[0])):
	str_column_to_float(dataset, i)
# Avaliação da performance do algoritmo
n_folds = 5
max_depth = 5
min_size = 10
scores = evaluate_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Scores: [96.35036496350365, 97.08029197080292, 97.44525547445255, 98.17518248175182, 97.44525547445255]
Mean Accuracy: 97.299%
